In [8]:
import firebase_admin
from firebase_admin import credentials, firestore
import json
import csv
from datetime import datetime 

# Path to your service account key file
service_account_path = 'sidewalk-survey-f7904-firebase-adminsdk-kgf2f-0493a854f1.json'

# Check if the Firebase app is already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

# For Firestore
db = firestore.client()

# Reference to the collection you want to download data from
# collection_name = 'surveyAnswers0727'
collection_name = 'surveyAnswers0807'
collection_ref = db.collection(collection_name)

# Load email addresses from the CSV file
email_file_path = 'valid-email-addresses.csv'
with open(email_file_path, newline='') as csvfile:
    email_reader = csv.reader(csvfile)
    email_addresses = [row[0] for row in email_reader]  # Assuming emails are in the first column

# Query documents where the email field matches any of the emails from the CSV file
data = []
for email in email_addresses:
    try:
        docs = collection_ref.where('email', '==', email).stream()
        for doc in docs:
            doc_data = doc.to_dict()
            # Convert Firestore document data to JSON serializable format
            for key, value in doc_data.items():
                if isinstance(value, datetime):
                    doc_data[key] = value.isoformat()
            data.append(doc_data)
    except Exception as e:
        print(f"Error accessing Firestore collection for email {email}: {e}")

# Check if data is actually collected
if not data:
    print("No data collected from Firestore.")

# Save data to a JSON file
# output_file_path = 'data/survey-responses/surveyAnswers0727_0827_download.json'
output_file_path = 'data/survey-responses/surveyAnswers0807_0827_download.json'
with open(output_file_path, 'w') as f:
    json.dump(data, f)

print(f"Data saved to {output_file_path}")

Data saved to data/survey-responses/surveyAnswers0807_0827_download.json
